In [1]:
pip install ipynb

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
from skimage import io
from tensorflow.keras import Input, layers
from tensorflow.keras import models
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.lite.experimental.microfrontend.python.ops import audio_microfrontend_op as frontend_op
print(tf.__version__)
import numpy as np
import shutil, os
import matplotlib.pyplot as plt
import pathlib
import random
import sys
import io
import os
import glob
from datetime import datetime as dt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, AveragePooling2D, MaxPooling2D
from tensorflow.keras.regularizers import l2
import os
from absl import app

assert tf.__version__.startswith('2')
%run functions.ipynb #pulling in functions from other file 

2.8.0


In [14]:
NUM_LABELS=2 #Binary classification so 2, [bird:0, unknown:1]
IMG_SIZE=IMAGE_SIZE=96 #image resolution is 96x96 dictated by pico
EPOCHS=100 #Number of epochs to train for
BATCH_SIZE = 64 #Batch for speed and regularization
INPUT_SHAPE= (IMG_SIZE,IMG_SIZE,1)
norm_layer = tf.keras.layers.Normalization(axis=-1)

In [23]:
BASE_DIR ="/Users/Jeromey/IOT/project2/data/augmented"
validation_split = 0.1
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=.1,
    horizontal_flip=True,
    validation_split=validation_split,
    #rescale=1. / 255
)
train_generator = datagen.flow_from_directory(
    BASE_DIR,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training',
    color_mode='grayscale') # was 'rgb'
val_generator = datagen.flow_from_directory(
    BASE_DIR,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation',
    color_mode='grayscale') # was 'rgb'
print(train_generator.class_indices)

Found 458517 images belonging to 2 classes.
Found 50945 images belonging to 2 classes.
{'notperson': 0, 'person': 1}


In [37]:
EPOCHS=50

In [38]:
model = models.Sequential([
    layers.Input(shape=INPUT_SHAPE),
 #   norm_layer,
    
    
    
    layers.Conv2D(8, 3,2, activation='relu'),
    layers.DepthwiseConv2D(kernel_size=(3,3), padding='same'),


                        
    layers.Conv2D(16, 3, activation='relu'),
    layers.DepthwiseConv2D(kernel_size=(3,3), padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.4), 
    
    layers.Conv2D(32, 3, activation='relu'),
    layers.DepthwiseConv2D(kernel_size=(3,3), padding='same'),
    #layers.MaxPooling2D(pool_size=(2, 2)),

                        
    layers.Conv2D(16, 3, activation='relu'),
    layers.DepthwiseConv2D(kernel_size=(3,3), padding='same'),
  #  layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(8, 3, activation='relu'),
    layers.DepthwiseConv2D(kernel_size=(3,3), padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),                   
    
    
    layers.Flatten(),

    layers.Dense(256),
    
    layers.Dense(2, activation='softmax'),
], name="attempt_1")
model.summary()


Model: "attempt_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 47, 47, 8)         80        
                                                                 
 depthwise_conv2d_20 (Depthw  (None, 47, 47, 8)        80        
 iseConv2D)                                                      
                                                                 
 conv2d_21 (Conv2D)          (None, 45, 45, 16)        1168      
                                                                 
 depthwise_conv2d_21 (Depthw  (None, 45, 45, 16)       160       
 iseConv2D)                                                      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 22, 22, 16)       0         
 2D)                                                             
                                                         

In [39]:
patience=5


steps=256
model.compile(
      optimizer=tf.keras.optimizers.Adam(),
      loss='categorical_crossentropy',
      metrics=['accuracy'])


callback=tf.keras.callbacks.EarlyStopping(
    monitor= 'val_accuracy',
    verbose=1,
    patience=15,
    restore_best_weights=True
)




In [40]:
history = model.fit(
    train_generator, 
    validation_data=val_generator,  
    steps_per_epoch=steps,
    validation_steps=steps,
    epochs=EPOCHS,
    callbacks=[callback]
)

Epoch 1/50
256/256 [==============================] - 38s 145ms/step - loss: 0.6413 - accuracy: 0.6036 - val_loss: 0.6332 - val_accuracy: 0.6278
Epoch 2/50
256/256 [==============================] - 37s 144ms/step - loss: 0.5182 - accuracy: 0.7330 - val_loss: 0.5927 - val_accuracy: 0.6757
Epoch 3/50
256/256 [==============================] - 37s 143ms/step - loss: 0.4766 - accuracy: 0.7601 - val_loss: 0.6245 - val_accuracy: 0.6805
Epoch 4/50
256/256 [==============================] - 37s 144ms/step - loss: 0.4615 - accuracy: 0.7678 - val_loss: 0.5917 - val_accuracy: 0.7017
Epoch 5/50
256/256 [==============================] - 37s 144ms/step - loss: 0.4374 - accuracy: 0.7850 - val_loss: 0.5824 - val_accuracy: 0.7234
Epoch 6/50
256/256 [==============================] - 37s 144ms/step - loss: 0.4255 - accuracy: 0.7941 - val_loss: 0.5808 - val_accuracy: 0.7269
Epoch 7/50
256/256 [==============================] - 37s 143ms/step - loss: 0.4110 - accuracy: 0.8065 - val_loss: 0.5747 - val_ac

In [42]:
test_model(100)

Bird accuracy: 0.83
Unknown accuracy: 0.76


In [ ]:
print("\n")
save_model()
print("\n")
plot_accuracy(history)
print("\n")
test_model(1000)

In [ ]:
test_model(10000)

In [ ]:
val_loss, val_acc = model.evaluate(validation_generator, verbose=2)

In [ ]:
BASE_DIR ="/Users/Jeromey/IOT/project2/data/train/p"

In [ ]:
# model = tf.keras.models.load_model(argv[1])
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
# with tf.io.gfile.GFile('vww_96_float.tflite', 'wb') as float_file:
#   float_file.write(tflite_model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset_gen():
  dataset_dir = os.path.join(BASE_DIR, "person")
  for idx, image_file in enumerate(os.listdir(dataset_dir)):
    # 10 representative images should be enough for calibration.
    if idx > 10000:
        return
    full_path = os.path.join(dataset_dir, image_file)
    if os.path.isfile(full_path):
      img = tf.keras.preprocessing.image.load_img(
          full_path, color_mode="grayscale").resize((96, 96))
      arr = tf.keras.preprocessing.image.img_to_array(img)
      # Scale input to [0, 1.0] like in training.
      yield [arr.reshape(1, 96, 96, 1)]
      #yield [arr.reshape(1, 96, 96, 1) / 255.] 

# Convert model to full-int8 and save as quantized tflite flatbuffer.
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
quantized_tflite_model = converter.convert()
with tf.io.gfile.GFile('vww_96_int8_SuperTry.tflite', 'wb') as quantized_file:
    quantized_file.write(quantized_tflite_model)

In [31]:
x1=tf.keras.utils.load_img(
    path='data/train/p/person/30000.jpeg',
    color_mode="grayscale",
    target_size=(IMG_SIZE,IMG_SIZE),
    interpolation='nearest'
)

In [32]:
x2=tf.keras.utils.load_img(
    path='data/train/n/notperson/30000.jpeg',
    color_mode="grayscale",
    target_size=(IMG_SIZE,IMG_SIZE),
    interpolation='nearest'
)

In [33]:
x1_arr=tf.keras.utils.img_to_array(x1)
x1_shape=x1_arr.reshape((1,) +x1_arr.shape)
x1_predict=model.predict(x1_shape)
x1_predict

array([[0.49701282, 0.5029872 ]], dtype=float32)

In [34]:
x2_arr=tf.keras.utils.img_to_array(x2)
x2_shape=x2_arr.reshape((1,) +x2_arr.shape)
x2_predict=model.predict(x2_shape)
x2_predict

array([[0.49853185, 0.5014681 ]], dtype=float32)

In [35]:
label_map = (train_generator.class_indices)

In [36]:
label_map

{'notperson': 0, 'person': 1}

In [ ]:
# model = tf.keras.models.load_model(argv[1])
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
# with tf.io.gfile.GFile('vww_96_float.tflite', 'wb') as float_file:
#   float_file.write(tflite_model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset_gen():
  dataset_dir = os.path.join(BASE_DIR, "person")
  for idx, image_file in enumerate(os.listdir(dataset_dir)):
    # 10 representative images should be enough for calibration.
    if idx > 10000:
        return
    full_path = os.path.join(dataset_dir, image_file)
    if os.path.isfile(full_path):
      img = tf.keras.preprocessing.image.load_img(
          full_path, color_mode="grayscale").resize((96, 96))
      arr = tf.keras.preprocessing.image.img_to_array(img)
      # Scale input to [0, 1.0] like in training.
      yield [arr.reshape(1, 96, 96, 1)]
      #yield [arr.reshape(1, 96, 96, 1) / 255.] 

# Convert model to full-int8 and save as quantized tflite flatbuffer.
converter.representative_dataset = representative_dataset_gen
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
quantized_tflite_model = converter.convert()
with tf.io.gfile.GFile('vww_96_flaot.tflite', 'wb') as quantized_file:
    quantized_file.write(quantized_tflite_model)